### 下載模組

在終端機輸入以下指令：

```
pip install git+https://github.com/ozgur/python-firebase
pip install selenium
pip install transformers
pip install -U ckip-transformers
```

CKIP 套件  
資料來源：https://github.com/ckiplab/ckip-transformers

### 匯入模組

In [2]:
import os
import base64
import random

# 函式註解
from typing import *

# 資料庫
from firebase import firebase

# 網路爬蟲
import requests
from time import sleep
from requests.exceptions import InvalidSchema
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from transformers import (
   BertTokenizerFast,
   AutoModelForMaskedLM,
   AutoModelForCausalLM,
   AutoModelForTokenClassification,
)
from ckip_transformers.nlp import CkipWordSegmenter

### 定義 Class MemeGenerator 

In [11]:
class MemeGenerator:
    def __init__(self, texts: List[str], url: str) -> None:
        self.url = url
        self.texts = texts

        self.chrome_options = Options()
        self.chrome_options.add_argument("--disable-gpu")

        # 無頭模式
        # self.chrome_options.add_argument("--headless")

    # 用自己的模型製作迷因
    def generate_meme(self) -> None:
        self.open_webdriver()
        self.type_texts()
        self.download_img(f"./pictures/result.png")
        self.close()

    # 用現成的 AI 製作迷因
    def generate_meme_predis_ai(self) -> None:
        # ckip 斷詞
        self.texts_preprocessing_with_ckip()

        # 至少 4 個詞彙才會執行
        if len(self.texts) >= 4:
            # 詞彙之間以空格做間隔
            self.texts = " ".join(self.texts)

            # 開始網路爬蟲
            self.open_webdriver()
            self.genrate_meme()
            self.close()
        else:
            print("字數過少")

    #### 預處理
    # 字數過長，則加上換行符號
    def texts_preprocessing(self) -> None:
        for index, text in enumerate(self.texts):
            if len(text) > 7:
                # 每 7 個字加上換行符號
                self.texts[index] = '\n'.join([text[i:i+7] for i in range(0, len(text), 7)])
    
    # ckip 斷詞
    def texts_preprocessing_with_ckip(self) -> None:
        # 以下三種模型擇一
        # masked language model (ALBERT, BERT)
        tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
        model = AutoModelForMaskedLM.from_pretrained('ckiplab/albert-tiny-chinese') # or other models above

        # casual language model (GPT2)
        # tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
        # model = AutoModelForCausalLM.from_pretrained('ckiplab/gpt2-base-chinese') # or other models above

        # nlp task model
        # tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
        # model = AutoModelForTokenClassification.from_pretrained('ckiplab/albert-tiny-chinese-ws') # or other models above

        # Initialize drivers
        # tokenizer
        ws_driver = CkipWordSegmenter(model = "bert-base")

        # 斷詞
        ws_result = ws_driver(self.texts)

        # 回傳斷詞結果，list[str]
        self.texts = ws_result[0]

    ##### 網路爬蟲 (https://imgflip.com/memetemplates)
    # 開啟瀏覽器
    def open_webdriver(self) -> None:
        # 初始化瀏覽器、設置智能等待
        # 注意!!! implicitly_wait 不要設定得太短
        self.driver = webdriver.Chrome('chromedriver', options = self.chrome_options)
        self.driver.implicitly_wait(20)

        # 開啟瀏覽器，並固定視窗大小
        self.driver.get(self.url)
        self.driver.set_window_size(1200, 800)
        sleep(0.5)

    # 輸入文本
    def type_texts(self) -> None:
        # 字數過長，則加上換行符號
        self.texts_preprocessing()

        parent = self.driver.find_element(By.CLASS_NAME, "mm-boxes")

        # 找到所有可填寫的欄位，並逐一填寫文本
        textareas = parent.find_elements(By.CLASS_NAME, "mm-text")
        for index, textarea in enumerate(textareas):
            textarea.send_keys(self.texts[index])

        sleep(0.5)

    # 下載圖片
    def download_img(self, image_path: str) -> None:
        # 點擊下載按鈕
        download_btn = self.driver.find_element(By.XPATH, r"/html/body/div[3]/div[2]/div[2]/div[10]/div[5]/button[1]")
        download_btn.click()

        # 找到圖片，並取得下載連結
        img = self.driver.find_element(By.ID, "done-img")
        img_url = img.get_attribute("src")
        sleep(0.5)

        # 解碼 Base64 圖片資料
        image_data = img_url.split(",")[1]
        image_bytes = base64.b64decode(image_data)

        # 輸出圖片
        with open(image_path, "wb") as file:
            file.write(image_bytes)
        
        sleep(0.5)

    ##### 網路爬蟲 (https://predis.ai/free-ai-tools/ai-meme-generator/#)
    def genrate_meme(self) -> None:
        # 輸入情境文本
        textarea = self.driver.find_element(By.CLASS_NAME, "MuiFilledInput-input")
        textarea.send_keys(self.texts)
        sleep(0.5)
        
        # 點擊 "GENERATE" 按鈕
        btn_genrate = self.driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div[3]/div/div/div/div/div[2]/div[2]/button[2]")
        btn_genrate.click()
        sleep(20)

        # 獲取圖片 url
        img_url = self.driver.find_element(By.CLASS_NAME, "MuiAvatar-img").get_attribute("src")
        print(img_url)

    # 關閉瀏覽器
    def close(self) -> None:
        self.driver.quit()

### 下載資料庫內容

In [4]:
DATABASE_URL = "https://fir-test-9907d-default-rtdb.firebaseio.com/"

fdb = firebase.FirebaseApplication(DATABASE_URL, None)
response = fdb.get("/meme_template", None)

### 主程式

##### 用自己的模型製作迷因

In [7]:
# 文本，預計用 AI 生成
texts = ["文本 1", "總字數超過7個字就換行", "文本 3", "文本 4", "文本 5", "文本 6", "文本 7", "文本 8"]

# 隨機抽取網址
urls = list(value["url"] for value in response.values())
url = random.choice(urls)

# 進行網路爬蟲，最終結果在 "pictures" 資料夾
MemeGen = MemeGenerator(texts, url)
MemeGen.generate_meme()

##### 用現成的 AI 製作迷因

In [12]:
texts = ["你們抓捕周樹人跟我魯迅有甚麼關係"]
url = "https://predis.ai/free-ai-tools/ai-meme-generator/#"

MemeGen = MemeGenerator(texts, url)
MemeGen.generate_meme_predis_ai()

Inference: 100%|██████████| 1/1 [00:00<00:00, 14.33it/s]


https://assets.predis.ai/calendar_user_upl_img/website.widget%40predis.ai/6475f2c128802c6b2d33e9c7_bab3241b-aa70-465e-a025-f0d7432c7d70_0_Kv4ox.jpeg
